In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import html, dcc, Input, Output, dash_table
import dash_bootstrap_components as dbc

In [13]:
def phasing_dimension():
    month = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
    return month

def sort_df_by_month(data_frame):
    month = phasing_dimension()
    data_frame['Mes'] = pd.Categorical(data_frame['Mes'], categories=month, ordered=True)
    data_frame = data_frame.sort_values('Mes')

    return data_frame

def current_year_data():
    data_excel = pd.ExcelFile('../data/Detalle_Gastos_Ganancias.xlsx')
    data = [pd.read_excel('../data/Detalle_Gastos_Ganancias.xlsx', sheet_name=sheet) for sheet in data_excel.sheet_names]
    for i in range(len(data)):
        if data[i][(data[i]['Categoria'] == 'Ingresos') & (data[i]['Mes'] == 'Enero')]['Cantidad ACT'].iloc[0] > 0:
            current_year = int(data_excel.sheet_names[i])
    return current_year

def previous_year_data():
    act_year = current_year_data()
    data_excel = pd.ExcelFile('../data/Detalle_Gastos_Ganancias.xlsx')
    if act_year == int(data_excel.sheet_names[0]):
        previous_year_data = None
    else:
        previous_year_data = act_year- 1
    return previous_year_data

def main_df():
    years = [previous_year_data(), current_year_data()]
    data = [pd.read_excel('../data/Detalle_Gastos_Ganancias.xlsx', sheet_name=str(sheet)) for sheet in years if sheet]
    for i in range(len(data)):
        data[i] = data[i][['Año', 'Mes', 'SubCategoria', 'Categoria', 'Cantidad ACT', 'Cantidad TGT', 'Cantidad PY']].fillna(0)
        data[i]['ACT vs TGT'] = data[i]['Cantidad ACT'] - data[i]['Cantidad TGT']
        data[i]['ACT vs PY'] = data[i]['Cantidad ACT'] - data[i]['Cantidad PY']
    df = pd.concat(data, ignore_index=True)
    return df

In [23]:
year=2024
data_frame = main_df()
df_prev = data_frame[(data_frame['Año'] == year)].groupby(['Categoria', 'Mes']).sum().reset_index()
df_prev = sort_df_by_month(df_prev)
df_prev = df_prev.pivot(index='Mes', columns='Categoria', values=['Cantidad ACT', 'Cantidad TGT', 'ACT vs TGT'])
df_prev = df_prev[df_prev['Cantidad ACT']['Ingresos']>0][['ACT vs TGT']]
df_prev = df_prev[:]['ACT vs TGT']
savings_per_month = []
for month in df_prev.index:
    savings_per_month.append(df_prev[:month].sum().sum())
savings_per_month

[1299.2399999999998]